<a href="https://colab.research.google.com/github/bonchevap/aigraz_project/blob/master/Notebooks/CleaningAndVisualization_LoadData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
import numpy as np
from sklearn.impute import SimpleImputer 

**Get the data from ENTSO-E platform**

In [3]:
! wget https://eepublicdownloads.blob.core.windows.net/public-cdn-container/clean-documents/Publications/Statistics/MHLV_data-2015-2017.xlsx
df=pd.read_excel("MHLV_data-2015-2017.xlsx")
df.head()

--2020-05-24 18:59:59--  https://eepublicdownloads.blob.core.windows.net/public-cdn-container/clean-documents/Publications/Statistics/MHLV_data-2015-2017.xlsx
Resolving eepublicdownloads.blob.core.windows.net (eepublicdownloads.blob.core.windows.net)... 52.239.248.196
Connecting to eepublicdownloads.blob.core.windows.net (eepublicdownloads.blob.core.windows.net)|52.239.248.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47495497 (45M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘MHLV_data-2015-2017.xlsx’

MHLV_data-2015-2017 100%[===================>]  45.29M  11.1MB/s    in 5.3s    

2020-05-24 19:00:05 (8.55 MB/s) - ‘MHLV_data-2015-2017.xlsx’ saved [47495497/47495497]



,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
0,Monthly Hourly Load Values,2014-12-31 23:00:00,2014-12-31,23:00:00,00:00:00,DE,98,46419.79,47367.132653
1,Monthly Hourly Load Values,2015-01-01 00:00:00,2015-01-01,00:00:00,01:00:00,DE,98,44898.30,45814.591837
2,Monthly Hourly Load Values,2015-01-01 01:00:00,2015-01-01,01:00:00,02:00:00,DE,98,43305.31,44189.091837
3,Monthly Hourly Load Values,2015-01-01 02:00:00,2015-01-01,02:00:00,03:00:00,DE,98,41918.17,42773.642857
4,Monthly Hourly Load Values,2015-01-01 03:00:00,2015-01-01,03:00:00,04:00:00,DE,98,41330.17,42173.642857


**We only need the data for Macedonia, which is country code MK**

In [4]:
df_MK=df[df.CountryCode == "MK"]
df_MK.head()

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
8789,Monthly Hourly Load Values,2015-12-31 23:00:00,2015-12-31,23:00:00,00:00:00,MK,100,1187.0,1187.0
8823,Monthly Hourly Load Values,2016-01-01 00:00:00,2016-01-01,00:00:00,01:00:00,MK,100,1142.0,1142.0
8858,Monthly Hourly Load Values,2016-01-01 01:00:00,2016-01-01,01:00:00,02:00:00,MK,100,1059.0,1059.0
8893,Monthly Hourly Load Values,2016-01-01 02:00:00,2016-01-01,02:00:00,03:00:00,MK,100,976.0,976.0
8928,Monthly Hourly Load Values,2016-01-01 03:00:00,2016-01-01,03:00:00,04:00:00,MK,100,929.0,929.0


**Save dayofweek, month, year, day, hour and add weekend feature in separate columns and drop some values**

In [0]:
df_MK=df_MK.assign(dayweek=df_MK.DateShort.dt.dayofweek)
df_MK=df_MK.assign(month=df_MK.DateShort.dt.month)
df_MK=df_MK.assign(year=df_MK.DateShort.dt.year)
df_MK=df_MK.assign(day=df_MK.DateShort.dt.day)
df_MK=df_MK.assign(hour=df_MK.DateUTC.dt.hour)
df_MK=df_MK.assign(weekend=df_MK.DateShort.dt.dayofweek >=5)
df_MK=df_MK.drop(["DateUTC", "DateShort", "TimeFrom", "TimeTo","CountryCode", "Cov_ratio", "MeasureItem", "Value_ScaleTo100" ], axis=1)


In [6]:
df_MK.head()

,Value,dayweek,month,year,day,hour,weekend
8789,1187.0,3,12,2015,31,23,False
8823,1142.0,4,1,2016,1,0,False
8858,1059.0,4,1,2016,1,1,False
8893,976.0,4,1,2016,1,2,False
8928,929.0,4,1,2016,1,3,False


**The data for 2018 and 2019 was not available on the above link and needed to be downloaded and read separately.**

In [0]:
df_MK_2018=pd.read_csv("Total Load - Day Ahead _ Actual_201801010000-201901010000.csv")
df_MK_2019=pd.read_csv("Total Load - Day Ahead _ Actual_201901010000-202001010000.csv")


**Renaming some of the columns. It can be noticed that in this dataset, apart from the actual load, the forecasted value for the electricity load in the day-ahead market has been also provided.**

In [8]:
df_MK_2018.head()
df_MK_2018.rename(columns={'Time (CET)': 'Time','Day-ahead Total Load Forecast [MW] - North Macedonia (MK)':'Load_Forecast(MW)','Actual Total Load [MW] - North Macedonia (MK)':'Load(MW)'  }, inplace=True)
df_MK_2018.head()

,Time,Load_Forecast(MW),Load(MW)
0,01.01.2018 00:00 - 01.01.2018 01:00,1010.0,923.0
1,01.01.2018 01:00 - 01.01.2018 02:00,941.0,886.0
2,01.01.2018 02:00 - 01.01.2018 03:00,858.0,821.0
3,01.01.2018 03:00 - 01.01.2018 04:00,799.0,766.0
4,01.01.2018 04:00 - 01.01.2018 05:00,769.0,727.0


In [9]:
df_MK_2019.head()
df_MK_2019.rename(columns={'Time (CET)': 'Time','Day-ahead Total Load Forecast [MW] - North Macedonia (MK)':'Load_Forecast(MW)','Actual Total Load [MW] - North Macedonia (MK)':'Load(MW)'  }, inplace=True)
df_MK_2019.head()

,Time,Load_Forecast(MW),Load(MW)
0,01.01.2019 00:00 - 01.01.2019 01:00,1118.0,1069.0
1,01.01.2019 01:00 - 01.01.2019 02:00,1066.0,1023.0
2,01.01.2019 02:00 - 01.01.2019 03:00,991.0,951.0
3,01.01.2019 03:00 - 01.01.2019 04:00,922.0,887.0
4,01.01.2019 04:00 - 01.01.2019 05:00,875.0,836.0


**Also, the time format is a little bit different than the 2016-2017 dataset. Some formatting is done:**

In [0]:
columns=['Time1', 'Time2']

df_MK_2018[columns]=df_MK_2018.Time.str.split('-', expand=True)
df_MK_2019[columns]=df_MK_2019.Time.str.split('-', expand=True)




In [0]:

df_MK_2018=df_MK_2018.drop( ["Time", "Time2"], axis=1)
df_MK_2019=df_MK_2019.drop( ["Time", "Time2"], axis=1)



**Converting the Time column to datetime and adding dayofweek, month, year, day, hour and weekend as separate columns.**

In [0]:

df_MK_2018['Time1'] = pd.to_datetime(df_MK_2018['Time1'])
df_MK_2019['Time1'] = pd.to_datetime(df_MK_2019['Time1'])


In [0]:


df_MK_2018=df_MK_2018.assign(dayweek=df_MK_2018.Time1.dt.dayofweek)
df_MK_2018=df_MK_2018.assign(month=df_MK_2018.Time1.dt.month)
df_MK_2018=df_MK_2018.assign(year=df_MK_2018.Time1.dt.year)
df_MK_2018=df_MK_2018.assign(day=df_MK_2018.Time1.dt.day)
df_MK_2018=df_MK_2018.assign(hour=df_MK_2018.Time1.dt.hour)
df_MK_2018=df_MK_2018.assign(weekend=df_MK_2018.Time1.dt.dayofweek >=5)
df_MK_2018=df_MK_2018.drop( "Time1", axis=1)

df_MK_2019=df_MK_2019.assign(dayweek=df_MK_2019.Time1.dt.dayofweek)
df_MK_2019=df_MK_2019.assign(month=df_MK_2019.Time1.dt.month)
df_MK_2019=df_MK_2019.assign(year=df_MK_2019.Time1.dt.year)
df_MK_2019=df_MK_2019.assign(day=df_MK_2019.Time1.dt.day)
df_MK_2019=df_MK_2019.assign(hour=df_MK_2019.Time1.dt.hour)
df_MK_2019=df_MK_2019.assign(weekend=df_MK_2019.Time1.dt.dayofweek >=5)
df_MK_2019=df_MK_2019.drop( "Time1", axis=1)


In [14]:
df_MK_2018.head()

,Load_Forecast(MW),Load(MW),dayweek,month,year,day,hour,weekend
0,1010.0,923.0,0,1,2018,1,0,False
1,941.0,886.0,0,1,2018,1,1,False
2,858.0,821.0,0,1,2018,1,2,False
3,799.0,766.0,0,1,2018,1,3,False
4,769.0,727.0,0,1,2018,1,4,False


In [15]:
df_MK_2019.head()

,Load_Forecast(MW),Load(MW),dayweek,month,year,day,hour,weekend
0,1118.0,1069.0,1,1,2019,1,0,False
1,1066.0,1023.0,1,1,2019,1,1,False
2,991.0,951.0,1,1,2019,1,2,False
3,922.0,887.0,1,1,2019,1,3,False
4,875.0,836.0,1,1,2019,1,4,False


**The final dataframe that we need includes the values from the 3 datasets.**

In [16]:

df_MK_final=df_MK_2018.append(df_MK_2019)
df_MK_final.head()
df_MK_withoutforecast=df_MK_final
df_MK_withoutforecast=df_MK_withoutforecast.drop( "Load_Forecast(MW)", axis=1)
df_MK_withoutforecast.head()





,Load(MW),dayweek,month,year,day,hour,weekend
0,923.0,0,1,2018,1,0,False
1,886.0,0,1,2018,1,1,False
2,821.0,0,1,2018,1,2,False
3,766.0,0,1,2018,1,3,False
4,727.0,0,1,2018,1,4,False


In [17]:
df_MK_withoutforecast.rename(columns={'Load(MW)': 'LoadMW'}, inplace=True)
df_MK.rename(columns={'Value': 'LoadMW'}, inplace=True)
df_MK_merged=df_MK.append(df_MK_withoutforecast)
df_MK_merged.head()


,LoadMW,dayweek,month,year,day,hour,weekend
8789,1187.0,3,12,2015,31,23,False
8823,1142.0,4,1,2016,1,0,False
8858,1059.0,4,1,2016,1,1,False
8893,976.0,4,1,2016,1,2,False
8928,929.0,4,1,2016,1,3,False


**There are some unreasonable values for the load data which are probably typo errors. They are fixed by dividing with 10,100 or  1000, depending on the range.**

In [18]:
df_rows4=df_MK_merged[(df_MK_merged.LoadMW >= 3000) & (df_MK_merged.LoadMW <=9999)].index
df_MK_merged.loc[df_rows4, 'LoadMW']=df_MK_merged.loc[df_rows4, 'LoadMW']/10
df_rows5=df_MK_merged[(df_MK_merged.LoadMW >= 10000) & (df_MK_merged.LoadMW <=99999)].index
df_MK_merged.loc[df_rows5, 'LoadMW']=df_MK_merged.loc[df_rows5, 'LoadMW']/100
df_rows6=df_MK_merged[(df_MK_merged.LoadMW >= 100000) & (df_MK_merged.LoadMW <=999999)].index
df_MK_merged.loc[df_rows6, 'LoadMW']=df_MK_merged.loc[df_rows6, 'LoadMW']/1000
df_MK_merged.head()

,LoadMW,dayweek,month,year,day,hour,weekend
8789,1187.0,3,12,2015,31,23,False
8823,1142.0,4,1,2016,1,0,False
8858,1059.0,4,1,2016,1,1,False
8893,976.0,4,1,2016,1,2,False
8928,929.0,4,1,2016,1,3,False


In [19]:
df_MK_merged.year.unique()

array([2015, 2016, 2017, 2018, 2019])

**Average hourly consumption (load) of the total consumption for 2017.**

In [20]:
#VISUALIZATION

#Average hourly consumption of the total consumption for 2017
df_2017=df_MK_merged[df_MK_merged.year==2017]
df_2017avr=df_2017.groupby(["month", "hour"]).LoadMW.mean().reset_index()
px.line(df_2017avr, x="hour", y="LoadMW", color="month")

**Average hourly consumption of the total consumption for 2016.**

In [21]:
#VISUALIZATION

#Average hourly consumption of the total consumption for 2016
df_2016=df_MK_merged[df_MK_merged.year==2016]
df_2016avr=df_2016.groupby(["month", "hour"]).LoadMW.mean().reset_index()
px.line(df_2016avr, x="hour", y="LoadMW", color="month")

**Average hourly consumption of the total consumption for 2018.**

In [22]:
#VISUALIZATION

#Average hourly consumption of the total consumption for 2018
df_2018=df_MK_merged[df_MK_merged.year==2018]
df_2018avr=df_2018.groupby(["month", "hour"]).LoadMW.mean().reset_index()
px.line(df_2018avr, x="hour", y="LoadMW", color="month")

**Average hourly consumption of the total consumption for 2019.**

In [23]:
#Average hourly consumption of the total consumption for 2019
df_2019=df_MK_merged[df_MK_merged.year==2019]
df_2019avr=df_2019.groupby(["month", "hour"]).LoadMW.mean().reset_index()
px.line(df_2019avr, x="hour", y="LoadMW", color="month")

**Average daily consumption of the total consumption for the years 2016, 2017, 2018, 2019.**

In [24]:
#Average daily consumption of the total consumption for the years 2016,2017,2018,2019
df_MK_year=df_MK_merged.groupby(["dayweek", "month", "year","hour"]).LoadMW.mean().reset_index()
df_MK_year.head()
df_MK_year_grouped=df_MK_year.groupby(["dayweek", "year", "hour"]).LoadMW.mean().reset_index()
df_MK_year_grouped.head()
df_MK_year_groupedf=df_MK_year_grouped.groupby(["dayweek", "year"]).LoadMW.sum().reset_index()
px.line(df_MK_year_groupedf, x="dayweek", y="LoadMW", color="year", title="Average daily consumption for the years 2016-2019")

In [0]:
df_MK_merged.to_csv("/content/MK_LoadData2016-2019.csv", index=False)

In [26]:
df_MK_final.head()
df_MK_final.describe()

,Load_Forecast(MW),Load(MW),dayweek,month,year,day,hour
count,17112.000000,16536.000000,17522.000000,17522.000000,17522.000000,17522.000000,17522.000000
mean,823.012272,853.518263,2.993494,6.526424,2018.500000,15.721892,11.498916
std,196.347830,8220.879089,2.001901,3.447953,0.500014,8.796898,6.922733
min,291.000000,0.000000,0.000000,1.000000,2018.000000,1.000000,0.000000
25%,700.000000,635.000000,1.000000,4.000000,2018.000000,8.000000,5.000000
50%,798.000000,751.000000,3.000000,7.000000,2018.500000,16.000000,11.000000
75%,950.000000,907.000000,5.000000,10.000000,2019.000000,23.000000,17.000000
max,1449.000000,997974.000000,6.000000,12.000000,2019.000000,31.000000,23.000000


In [27]:
df_MK_final.rename(columns={'Load(MW)': 'LoadMW'}, inplace=True)
df_rows4N=df_MK_final[(df_MK_final.LoadMW >= 3000) & (df_MK_final.LoadMW <=9999)].index
df_MK_final.loc[df_rows4, 'LoadMW']=df_MK_final.loc[df_rows4N, 'LoadMW']/10
df_rows5N=df_MK_final[(df_MK_final.LoadMW >= 10000) & (df_MK_final.LoadMW <=99999)].index
df_MK_final.loc[df_rows5N, 'LoadMW']=df_MK_final.loc[df_rows5N, 'LoadMW']/100
df_rows6N=df_MK_final[(df_MK_final.LoadMW >= 100000) & (df_MK_final.LoadMW <=999999)].index
df_MK_final.loc[df_rows6N, 'LoadMW']=df_MK_final.loc[df_rows6N, 'LoadMW']/1000
df_MK_final.describe()

,Load_Forecast(MW),LoadMW,dayweek,month,year,day,hour
count,17112.000000,16536.000000,17522.000000,17522.000000,17522.000000,17522.000000,17522.000000
mean,823.012272,771.956825,2.993494,6.526424,2018.500000,15.721892,11.498916
std,196.347830,214.905588,2.001901,3.447953,0.500014,8.796898,6.922733
min,291.000000,0.000000,0.000000,1.000000,2018.000000,1.000000,0.000000
25%,700.000000,635.000000,1.000000,4.000000,2018.000000,8.000000,5.000000
50%,798.000000,751.000000,3.000000,7.000000,2018.500000,16.000000,11.000000
75%,950.000000,907.000000,5.000000,10.000000,2019.000000,23.000000,17.000000
max,1449.000000,1423.000000,6.000000,12.000000,2019.000000,31.000000,23.000000


In [0]:
df_MK_final.rename(columns={'Load_Forecast(MW)': 'ForecastMW'}, inplace=True)

In [29]:
df_MK_final.head()

,ForecastMW,LoadMW,dayweek,month,year,day,hour,weekend
0,1010.0,923.0,0,1,2018,1,0,False
1,941.0,886.0,0,1,2018,1,1,False
2,858.0,821.0,0,1,2018,1,2,False
3,799.0,766.0,0,1,2018,1,3,False
4,769.0,727.0,0,1,2018,1,4,False


**Saving the load data ONLY, for 2018-2019 in a separate file.**

In [0]:
df_MK_final.to_csv("/content/MK_ForecastDataandLoadData2018-2019.csv", index=False)

In [31]:
df_MK_forecastonly=df_MK_final.drop("LoadMW", axis=1)
df_MK_forecastonly.head()

,ForecastMW,dayweek,month,year,day,hour,weekend
0,1010.0,0,1,2018,1,0,False
1,941.0,0,1,2018,1,1,False
2,858.0,0,1,2018,1,2,False
3,799.0,0,1,2018,1,3,False
4,769.0,0,1,2018,1,4,False


**Saving the forecasted data ONLY, for 2018-2019.**

In [0]:
df_MK_forecastonly.to_csv("/content/MK_ForecastDataOnly2018-2019.csv", index=False)

In [33]:
df_MK_test=df_MK_merged[df_MK_merged.year==2019]
df_MK_test.describe()

,LoadMW,dayweek,month,year,day,hour
count,8016.000000,8761.000000,8761.000000,8761.0,8761.000000,8761.000000
mean,779.476724,2.994864,6.526424,2019.0,15.721835,11.498916
std,210.231254,2.000250,3.448051,0.0,8.797072,6.922931
min,0.420000,0.000000,1.000000,2019.0,1.000000,0.000000
25%,637.000000,1.000000,4.000000,2019.0,8.000000,5.000000
50%,759.000000,3.000000,7.000000,2019.0,16.000000,11.000000
75%,902.000000,5.000000,10.000000,2019.0,23.000000,17.000000
max,1423.000000,6.000000,12.000000,2019.0,31.000000,23.000000


In [0]:
df_MK_test.to_csv("/content/MK_test2019.csv", index=False)